In [1]:
import numpy as np
import pandas as pd

In [2]:
# reading csv of all candidates declared as rep or dem
candidate_df = pd.read_csv("candidatelist.csv", header=2)

In [3]:
candidate_df.head()

,First Name,Last Name,Address,City,Zip,DO,Office Title,Pseudo,District,Party,Waiver,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Office ID
0,Dom,Costa,117 Hawthorne Crt.,Pittsburgh,15201,0,Sheriff,679,COUNTYWIDE,Democratic,False,False,188,10,7,4,1,COUNTYWIDE,COUNTY,520
1,Kevin,Kraus,156 Berkshire Court,Pittsburgh,15237,0,Sheriff,523,COUNTYWIDE,Democratic,False,True,183,0,3,4,1,COUNTYWIDE,COUNTY,520
2,Joe,Wise,1851 Madison Dr.,Moon,15108,0,Member of County Council,418,DISTRICT 1,Republican,True,False,172,0,11,4,1,COUNTY COUNCIL,CNTCL01,530
3,Jack,Betkowski,113 Ambleside Dr.,Pittsburgh,15237,0,Member of County Council,1079,DISTRICT 1,Democratic,True,False,196,9,3,4,1,COUNTY COUNCIL,CNTCL01,530
4,Robert J.,Palmosina,2579 Greenboro Lane,Pittsburgh,15220,0,Member of County Council,888,DISTRICT 12,Democratic,False,False,188,20,16,4,1,COUNTY COUNCIL,CNTCL12,530


In [4]:
candidate_df["Office Title"].value_counts()

Member of Council             330
School Director               282
Judge of Election             107
Mayor                          90
Constable                      83
Tax Collector                  82
Inspector of Election          79
Magisterial District Judge     72
Commissioner                   64
Supervisor                     24
School Director At-Large       12
Member of County Council       11
Auditor                         7
Member of Council At-Large      4
Sheriff                         2
Controller                      2
Commissioner At-Large           2
Treasurer                       1
Name: Office Title, dtype: int64

In [5]:
candidate_df[candidate_df["Office Title"].isin(["School Director","School Director At-Large"])]

,First Name,Last Name,Address,City,Zip,DO,Office Title,Pseudo,District,Party,Waiver,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Office ID
23,Sylvia C.,Wilson,P.O.Box 4812,Pittsburgh,15206,0,School Director,714,DISTRICT 1,Republican,False,False,188,12,5,4,1,CITY SCHOOLS,PGHSCH1,710
24,Grace,Higginbotham,7103 Kedron St.,Pittsburgh,15208,0,School Director,726,DISTRICT 1,Democratic,False,False,188,13,1,4,1,CITY SCHOOLS,PGHSCH1,710
25,Carlos A.,Thomas,7152 Upland St.,Pittsburgh,15208,0,School Director,729,DISTRICT 1,Democratic,True,False,188,13,4,4,1,CITY SCHOOLS,PGHSCH1,710
26,Sylvia C.,Wilson,P.O.Box 4812,Pittsburgh,15206,0,School Director,714,DISTRICT 1,Democratic,False,False,188,12,5,4,1,CITY SCHOOLS,PGHSCH1,710
27,Lamont,"Frazier, Jr.",5161 Hillcrest St.,Pittsburgh,15224,0,School Director,691,DISTRICT 3,Democratic,False,False,188,10,19,4,1,CITY SCHOOLS,PGHSCH3,710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,Todd,Kosenina,47 Trotter Ln.,Clinton,15026,0,School Director,204,WEST ALLEGHENY REGION 3,Republican,True,False,141,0,1,4,1,SCH/REGION,SCH0070,1000
385,Todd,Kosenina,47 Trotter Ln.,Clinton,15026,0,School Director,204,WEST ALLEGHENY REGION 3,Democratic,True,False,141,0,1,4,1,SCH/REGION,SCH0070,1000
386,Steven Russell,Pettit,420 Colt Cir,Clinton,15026,0,School Director,206,WEST ALLEGHENY REGION 3,Democratic,True,False,141,0,3,4,1,SCH/REGION,SCH0070,1000
387,Charles B.,Wade,"531 Park Ave,",Clariton,15025,0,School Director,126,CLAIRTON REGION 4,Democratic,True,False,123,4,2,4,1,SCH/REGION,SCH0012,1010


# Taking a look at School Directors
## Data Cleaning

In [6]:
school_df = candidate_df[candidate_df["Office Title"].isin(["School Director","School Director At-Large"])]

In [7]:
#lets not bother w pittsburgh
school_df = school_df[school_df.City!="Pittsburgh"]

In [8]:
school_df["District"].value_counts()

MOON AREA                  26
GATEWAY                    12
SOUTH ALLEGHENY            10
DEER LAKES                 10
RIVERVIEW                   9
MCKEESPORT AREA             8
SOUTH FAYETTE               6
MONTOUR REGION 3            6
STEEL VALLEY                6
ALLEGHENY VALLEY            5
SOUTH PARK                  5
PINE RICHLAND REGION 2      4
QUAKER VALLEY REGION 1      4
CLAIRTON                    4
WEST ALLEGHENY REGION 2     4
NORTH ALLEGHENY             4
AVONWORTH                   4
SHALER AREA REGION 3        4
WEST ALLEGHENY REGION 3     4
QUAKER VALLEY REGION 3      3
STO-ROX REGION 1            3
DUQUESNE                    3
WEST MIFFLIN AREA           2
STO-ROX                     2
EAST ALLEGHENY REGION 2     2
CHARTIERS VALLEY            2
PINE RICHLAND REGION 1      2
PINE RICHLAND REGION 3      2
COUNTYWIDE                  2
PLUM                        2
CLAIRTON REGION 4           1
STO-ROX REGION 2            1
STEEL VALLEY REGION 2       1
CLAIRTON R

In [9]:
#????????? what
school_df[school_df["District"]=="COUNTYWIDE"]

,First Name,Last Name,Address,City,Zip,DO,Office Title,Pseudo,District,Party,Waiver,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Office ID
355,Tyler,Kochirka,1530 Glenn Way,McKees Rocks,15136,0,School Director,1177,COUNTYWIDE,Democratic,True,False,209,4,1,2,1,COUNTYWIDE,COUNTY,980
388,Tisha,Thomas,903 Derby Ln.,N. Versailles,15137,0,School Director,502,COUNTYWIDE,Democratic,True,False,179,7,0,4,1,COUNTYWIDE,COUNTY,1040


In [10]:
school_df = school_df[school_df["District"]!="COUNTYWIDE"]

In [11]:
# this is way too many columns
school_df.head()

,First Name,Last Name,Address,City,Zip,DO,Office Title,Pseudo,District,Party,Waiver,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Office ID
110,William Robert,McLaughlin,3461 PinewoodDr.,West Homestead,15120,0,School Director At-Large,1244,STEEL VALLEY,Republican,True,False,221,0,3,4,3,SCHOOL,SCH0063,890
111,Cara Perhacs,Karstetter,3025 Alberta St.,Munhall,15120,0,School Director At-Large,468,STEEL VALLEY,Republican,True,False,175,0,6,4,3,SCHOOL,SCH0063,890
112,Mary,Yuhas,158 W. Marigold St.,Munhall,15120,0,School Director At-Large,471,STEEL VALLEY,Republican,True,False,175,0,9,4,3,SCHOOL,SCH0063,890
113,Mary,Yuhas,158 W. Marigold St.,Munhall,15120,0,School Director At-Large,471,STEEL VALLEY,Democratic,True,False,175,0,9,4,3,SCHOOL,SCH0063,890
114,Cara Perhacs,Karstetter,3025 Alberta St.,Munhall,15120,0,School Director At-Large,468,STEEL VALLEY,Democratic,True,False,175,0,6,4,3,SCHOOL,SCH0063,890


In [12]:
school_indices = list(school_df.index)

In [13]:
school_cleaned = pd.DataFrame()

In [14]:
name = []
party = []
district = []
for i in school_indices:
    row = school_df[school_df.index==i]
    name.append(row["First Name"].to_string(index=False) + " " + row["Last Name"].to_string(index=False))
    party.append(row["Party"].to_string(index=False))
    district.append(row["District"].to_string(index=False))

In [15]:
district2 = []
ward = []
for x in district:
    if "REGION" in x:
        parts = x.split("REGION")
        ward.append(int(parts[-1]))
        district2.append(' '.join(parts[:-1]).strip())
    else:
        ward.append(np.nan)
        district2.append(x)
    

In [16]:
school_cleaned["name"] = name
school_cleaned["party"] = party
school_cleaned["district"] = district2
school_cleaned["ward"] = ward

In [17]:
school_cleaned.head()

,name,party,district,ward
0,William Robert McLaughlin,Republican,STEEL VALLEY,NaN
1,Cara Perhacs Karstetter,Republican,STEEL VALLEY,NaN
2,Mary Yuhas,Republican,STEEL VALLEY,NaN
3,Mary Yuhas,Democratic,STEEL VALLEY,NaN
4,Cara Perhacs Karstetter,Democratic,STEEL VALLEY,NaN


In [18]:
school_pos_df = pd.read_csv("schoolpos.csv")

In [19]:
school_pos_df.columns = ["num","district","term","ward"]

In [20]:
school_pos_df.head()

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4-year,NaN
1,4,AVONWORTH,4-year,NaN
2,4,BALDWIN-WHITEHALL,4-year,NaN
3,4,BETHEL PARK,4-year,NaN
4,4,BRENTWOOD,4-year,NaN


In [21]:
school_pos_df.term = school_pos_df.term.map(lambda x: int(x.split("-")[0]))

In [22]:
school_pos_df.head()

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4,NaN
1,4,AVONWORTH,4,NaN
2,4,BALDWIN-WHITEHALL,4,NaN
3,4,BETHEL PARK,4,NaN
4,4,BRENTWOOD,4,NaN


In [23]:
school_pos_cleaned = school_pos_df[school_pos_df.ward.notna()==False]

In [24]:
school_pos_cleaned.head()

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4,NaN
1,4,AVONWORTH,4,NaN
2,4,BALDWIN-WHITEHALL,4,NaN
3,4,BETHEL PARK,4,NaN
4,4,BRENTWOOD,4,NaN


In [25]:
ward_df = school_pos_df[school_pos_df.ward.notna()==True]

In [26]:
row_list = []
for row in range(len(ward_df)):
    regions = ward_df.iloc[[row]].ward.values[0].split(",")
    if len(regions)>1:
        for x in regions:
            temp_dict = {
                "num": 1, 
                "district": ward_df.iloc[row,1],
                "term": ward_df.iloc[row,2],
                "ward": x
            }
            row_list.append(temp_dict)  
    else:
        temp_dict = {
            "num": ward_df.iloc[row,0], 
            "district": ward_df.iloc[row,1],
            "term": ward_df.iloc[row,2],
            "ward": ward_df.iloc[row,3]
            }
        row_list.append(temp_dict)  

In [27]:
school_pos_cleaned = school_pos_cleaned.append(row_list)

In [28]:
school_pos_cleaned.ward.value_counts()

1           15
3           14
2           12
at-large     2
9            2
4            2
7            2
5            1
Name: ward, dtype: int64

In [29]:
school_pos_cleaned

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4,NaN
1,4,AVONWORTH,4,NaN
2,4,BALDWIN-WHITEHALL,4,NaN
3,4,BETHEL PARK,4,NaN
4,4,BRENTWOOD,4,NaN
...,...,...,...,...
45,2,WEST ALLEGHENY,4,2
46,1,WOODLAND HILLS,4,1
47,1,WOODLAND HILLS,4,2
48,2,WOODLAND HILLS,4,3


## Data Analysis

In [30]:
school_cleaned.head()

,name,party,district,ward
0,William Robert McLaughlin,Republican,STEEL VALLEY,NaN
1,Cara Perhacs Karstetter,Republican,STEEL VALLEY,NaN
2,Mary Yuhas,Republican,STEEL VALLEY,NaN
3,Mary Yuhas,Democratic,STEEL VALLEY,NaN
4,Cara Perhacs Karstetter,Democratic,STEEL VALLEY,NaN


In [31]:
school_cleaned.district.value_counts()

MOON AREA            26
GATEWAY              12
DEER LAKES           10
SOUTH ALLEGHENY      10
RIVERVIEW             9
WEST ALLEGHENY        9
PINE RICHLAND         8
MCKEESPORT AREA       8
STEEL VALLEY          8
CLAIRTON              7
QUAKER VALLEY         7
STO-ROX               6
MONTOUR               6
SOUTH FAYETTE         6
SOUTH PARK            5
ALLEGHENY VALLEY      5
NORTH ALLEGHENY       4
SHALER AREA           4
AVONWORTH             4
EAST ALLEGHENY        3
DUQUESNE              3
CHARTIERS VALLEY      2
WEST MIFFLIN AREA     2
PLUM                  2
Name: district, dtype: int64

In [32]:
school_pos_cleaned.district.value_counts()

STO-ROX                 5
PITTSBURGH              5
EAST ALLEGHENY          5
CLAIRTON                5
WOODLAND HILLS          4
FOX CHAPEL AREA         4
KEYSTONE OAKS           3
HIGHLANDS               3
WEST ALLEGHENY          3
SHALER AREA             3
QUAKER VALLEY           3
PINE-RICHLAND           3
SOUTH ALLEGHENY         2
BRENTWOOD               2
CHARTIERS VALLEY        2
MONTOUR                 2
STEEL VALLEY            2
SOUTH PARK              1
SOUTH FAYETTE           1
MCKEESPORT AREA         1
WILKINSBURG             1
PENN HILLS              1
CORNELL                 1
ELIZABETH-FORWARD       1
WEST JEFFERSON HILLS    1
MOON AREA               1
RIVERVIEW               1
WEST MIFFLIN AREA       1
ALLEGHENY VALLEY        1
NORTHGATE               1
DEER LAKES              1
CARLYNTON               1
PLUM                    1
NORTH ALLEGHENY         1
BETHEL PARK             1
HAMPTON                 1
NORTH HILLS             1
BALDWIN-WHITEHALL       1
GATEWAY     